In [1]:
import json

In [2]:
metadata = json.load(open('instagram-images.json'))
mturk_results = json.load(open('mturk-results.json'))

In [3]:
prediction_files = ['predictions/AWS-Rekognition.json',
                    'predictions/Clarifai.json',
                    'predictions/Google-Cloud-Vision.json',
                    'predictions/Mobius-Labs.json'] 

In [4]:
results = dict()
for file_path in prediction_files:
    predictions = json.load(open(file_path))
    name = file_path.split('/')[-1].replace('.json','')    
    results[name] = {'true_postive':0,
                    'false_positve':0,
                    'total':0,
                    'unique_concepts':set()}
    for photo_id, concepts in predictions.items():
        for concept in concepts:
            if concept != concept.encode('ascii', 'ignore'):
                print u'skipping unicode concept \'{}\' for image {}'.format(concept, photo_id)
                continue
            if concept in mturk_results[photo_id]['correct']:
                results[name]['true_postive'] +=1
                results[name]['total'] +=1                
                results[name]['unique_concepts'].add(concept)
            elif concept in mturk_results[photo_id]['incorrect']:
                results[name]['false_positve'] +=1
                results[name]['total'] +=1  
                results[name]['unique_concepts'].add(concept)                
            else:
                raise ValueError('Found concept {} for {} for which don\'t have mturk data'.format(concept, photo_id))

skipping unicode concept 'fête' for image 1848089575245540899
skipping unicode concept 'fête' for image 1848089554164934063
skipping unicode concept 'château' for image 1847964988017179597


In [5]:
print 'Name             Labels/Image   Precision   Unique Concepts'
for name in sorted(results.keys()):
    pretty_name = name.replace('-', ' ')
    numbers = results[name]
    labels_per_image = round(numbers['total']/float(len(metadata)), 1)
    precision = round(numbers['true_postive']/float((numbers['total'])),3)
    unique_labels = len(numbers['unique_concepts'])
    print "{:21}{: 8.2f}{: 12.2f}{: 18d}".format(pretty_name, labels_per_image, precision, unique_labels)

Name             Labels/Image   Precision   Unique Concepts
AWS Rekognition         13.80        0.81              1026
Clarifai                20.00        0.84              1467
Google Cloud Vision      8.70        0.92              1140
Mobius Labs             16.30        0.92              1018
